<a href="https://colab.research.google.com/github/rafael100495/Aulas_Mackenzie/blob/main/BIG_A1_LinearRegressionStatsmodels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<head>
  <meta name="author" content="Rogério de Oliveira">
  <meta institution="author" content="Universidade Presbiteriana Mackenzie">
</head>

<img src="http://meusite.mackenzie.br/rogerio/mackenzie_logo/UPM.2_horizontal_vermelho.jpg" width=300, align="right">
<!-- <h1 align=left><font size = 6, style="color:rgb(200,0,0)"> optional title </font></h1> -->


# **Tarefa de Aprofundamento 1** 
## CASO: Estimando o Preço de Imóveis em São Paulo
---
<br>
<br>





A partir deste notebook crie um modelo de regressão múltipla com a biblioteca `statsmodels` para predição de preço de imóveis com base no histórico de preços e características de vários imóveis de São Paulo. 

Em seguida, com base nos seus resultados, responda o **questionário do Moodle**. 

# Basic imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# Exploração dos Dados

Entenda os dados disponíveis e em seguida faça as transformações necessárias nos dados. Quantidades dos dados, se existem valores nulos e valores máximos e mínimos dos atributos são algumas explorações comuns. 

(Moodle) **Q1**. Quantos bairros existem no histórico e qual o bairro mais presente nos dados?



In [ ]:
df = pd.read_excel('http://meusite.mackenzie.br/rogerio/data_load/regressao_preco_imoveis.xlsx')
df.head()

,bairro,areaM2,suites,dormitorios,banheiros,vagas,preco
0,vila-nova-conceicao,32,1,1,1,1,490000
1,vila-nova-conceicao,157,2,2,2,2,3180000
2,vila-nova-conceicao,205,2,3,3,3,1900000
3,vila-nova-conceicao,193,3,3,3,3,3565000
4,vila-nova-conceicao,116,1,3,2,2,1605000


In [79]:
qtdBairros = df.bairro.drop_duplicates()
print(f'A quantidade de bairros que existem são {len(qtdBairros)}')

A quantidade de bairros que existem são 5


In [80]:
# O bairro mais presente na tabela é o vila-mariana
bairroPresente = df.bairro.value_counts()
bairroPresente

vila-mariana           1714
paraiso                 802
vila-nova-conceicao     789
vila-sonia              307
morumbi                 129
Name: bairro, dtype: int64

# Preparação dos Dados

Use [statsmodels.formula.api](https://www.statsmodels.org/dev/generated/statsmodels.formula.api.ols.html)

```
from statsmodels.formula.api import ols   
```

Se você empregar esta forma do `statsmodels` ele fará de modo implítico o hot encode dos dados e não há necessidade de nenhuma transformação dos dados (não faremos a normalização aqui. Embora sempre recomendável na maior parte dos casos, aqui o resultado de predição será o mesmo com o sem a normalização).

Atenção, se você empregar 
```
from statsmodels.api import OLS 
```
os resultados serão os mesmos, mas alguma preparação nos dados será necessária e a construção do modelo será um pouco diferente.

In [ ]:
from statsmodels.formula.api import ols   

# nada a fazer se estiver empregando a api acima

# Estimando o modelo com o `statsmodels`

Estime o modelo.

(Moodle) **Q2**. Qual o coeficiente de determinação ajustado obtido?

Dica: empregue `result.rsquared_adj` no lugar de `result.summary()` para obter resultados com maior precisão.

(Moodle) **Q3**. Qual coeficiente estimado é menos significativo?

In [81]:
import statsmodels.formula.api as sm 
print(f'O coeficiente ajustado obtido é de 79,2% de resposta com o modelo.')
print(f'O coeficiente estimago menos significativo é a quantidade de banheiros, ela não interfere no preço.')

model_1 = sm.ols(formula = "preco ~ vagas + bairro + areaM2 + suites", data=df)

result_1 = model_1.fit()
print(result_1.summary())

O coeficiente ajustado obtido é de 79,2% de resposta com o modelo.
O coeficiente estimago menos significativo é a quantidade de banheiros, ela não interfere no preço.
                            OLS Regression Results                            
Dep. Variable:                  preco   R-squared:                       0.793
Model:                            OLS   Adj. R-squared:                  0.792
Method:                 Least Squares   F-statistic:                     2041.
Date:                Sun, 30 Oct 2022   Prob (F-statistic):               0.00
Time:                        19:43:31   Log-Likelihood:                -54313.
No. Observations:                3741   AIC:                         1.086e+05
Df Residuals:                    3733   BIC:                         1.087e+05
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                                    coef   

# Re-estimando o modelo com o `statsmodels`

Estime novamente o modelo excluindo o atributo menos significativo (um único atributo). 

(Moodle) **Q4**. Qual o novo coeficiente de determinação ajustado obtido?


In [82]:
print(f'O atributo menos significativo removido foi a suite, que afetou menos no modelo.')
print(f'O novo coeficiente obtido é de 78,5% de resposta com este modelo.')
import statsmodels.formula.api as sm 

model_2 = sm.ols(formula = "preco ~ vagas + bairro + areaM2", data=df)

result_2 = model_2.fit()
print(result_2.summary())

O atributo menos significativo removido foi a suite, que afetou menos no modelo.
O novo coeficiente obtido é de 78,5% de resposta com este modelo.
                            OLS Regression Results                            
Dep. Variable:                  preco   R-squared:                       0.785
Model:                            OLS   Adj. R-squared:                  0.785
Method:                 Least Squares   F-statistic:                     2270.
Date:                Sun, 30 Oct 2022   Prob (F-statistic):               0.00
Time:                        19:44:02   Log-Likelihood:                -54383.
No. Observations:                3741   AIC:                         1.088e+05
Df Residuals:                    3734   BIC:                         1.088e+05
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                                    coef    std err          t 

# Predição

Faça a predição do preço, com o melhor dos modelos obtidos acima, de um imóvel na Vila Mariana, com 134 m2, 1 suite, 4 dorm, 3 banheiros e 3 vagas.



(Moodle) **Q5**. Qual o preço estimado, para o melhor dos modelos obtidos, de um imóvel na Vila Mariana, com 134 m2, 1 suite, 4 dorm, 3 banheiros e 3 vagas?



In [84]:
model_3 = sm.ols(formula = "preco ~ bairro + areaM2 + suites + dormitorios + banheiros + vagas", data=df)
result_3 = model_3.fit()

x = pd.DataFrame({'bairro': ['vila-mariana'],'areaM2': [134], 'suites': [1], 'dormitorios': [4], 'banheiros': [3], 'vagas': [3]})
result_3.predict(x)
print('Montei um novo modelo com todas as variáveis solicitadas, e pelo que entendi, o valor é em torno de 1 milhão e 349 mil.')

Montei um novo modelo com todas as variáveis solicitadas, e pelo que entendi, o valor é em torno de 1 milhão e 349 mil.
